# Image plates of duckweed in growth assay experiment

In [1]:
#Importing python libraries downloaded from the internet and those saved locally
import serial 
from serial.tools import list_ports
import yaml
import matplotlib
matplotlib.use('TkAgg') #This is a specific back end of matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2 as cv2
import time
import pandas as pd
import glob
import json
import datetime
from datetime import date
from utils.MachineUtils import * #Local library
from utils.CameraUtils import * #Local library. #Note that this library requirs a package that can only be installed on raspbery pi. 
import utils.Plate_positions as pp #Local library

import ipywidgets as widgets
# from Ipython.display import display, Image
import threading

### 1. Define directories and specify plates to image
Ensure the following before running anything in this notebook:
 - Plates with duckweed are loaded in the correct positions on the bedplate
 - The machine is switched on, connected to the computer and all axes have been homed. 
 - The camera tool is accessible and in position 1 on the Jubilee

In [ ]:
#Define directory to save image data for this experiment
data_dir = '/home/pi/Documents/Data'
expt_dir = data_dir + '/Test_expt'
expt_id = "test"
print("Image files will be saved to:")
print(expt_dir)


### CELLS TO RUN FOR MANUAL PLATE SELECTION

In [ ]:
# Manual option: Which plates do you want to image (numbers correspond to positions on bedplate. Options: 1,2,3,4,5)
plates_to_image = [1]




### CELLS TO RUN FOR AUTOMATED PLATE SELECTION

In [ ]:

#Automated option: Define experiment set up file and read from that which plates are to be imaged. 

expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "TestExptDir" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_file_name = "TestFile.json" #Name for the experiment setup file (Metadata)

exp_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
print(exp_setup_file_path)

In [ ]:
os.chdir(exp_setup_file_path)

with open(expt_setup_file_name) as datafile:
    expt_data = json.load(datafile)

# Turn samples list into a dataframe
sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)

unique_plates = list(df.Plate.unique())

def pull_last_number(n):
    return n[-1:]

plates_to_image = list(map(pull_last_number, unique_plates))
print(plates_to_image)


### Step 2: Run these cells to define methods

In [ ]:
def move_across_platewells(well_coord_list_of_dicts):
    for i, well in enumerate(well_coord_list_of_dicts):
        m.moveTo(x=well["x"], y=well["y"], z=10)
        ser.write(b'G4 P1000') 

def image_plate(plate_num):
    well_coord_list_of_dicts = pp.fetch_plate_wellpostions(plate_num)
    for i, well in enumerate(well_coord_list_of_dicts):
        m.moveTo(x=well["x"], y=well["y"], z=10)
#         wellFrame, wellData = getSingleWell(getFrame(), minR=330, maxR=335) #Adjust focus
#         well = wellData[0]
#         print(well)
        f = getFrame()
        cv2.imwrite(f'{expt_dir}/{expt_id}_plate{plate_num}_well{well["well_id"]}_{date.today()}.jpg', f)

In [2]:
ports = serial.tools.list_ports.comports()
ser = serial.Serial('/dev/ttyACM0', 115200) #Port address and Baudrate (info transfer rate)
port = '/dev/ttyACM0'
m = MachineCommunication(port)
ser.write(b'G90\n') #Command to make sure the Jubilee is using absolute as opposed to relative positioning
m.toolChange(1)

### Step 3: Run this cell to image the plates defined in step 1 

In [ ]:
for p in plates_to_image:
    image_plate(p)

In [6]:
i = getCameraIndices()
print(i)

[0]


[ WARN:1@427.558] global /tmp/pip-wheel-8c7uejek/opencv-python_88dbbad412c5416b992ae69de26299d6/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ WARN:1@427.558] global /tmp/pip-wheel-8c7uejek/opencv-python_88dbbad412c5416b992ae69de26299d6/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ WARN:1@427.559] global /tmp/pip-wheel-8c7uejek/opencv-python_88dbbad412c5416b992ae69de26299d6/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video3): can't open camera by index


In [4]:
cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

success, img = cap.read()
cv2.imshow("Webcam", img)

# while True:
#     success, img = cap.read()
#     cv2.imshow("Webcam", img)
#     if cv2.waitKey(1) & OxFF==ord('q'):
#         cap.release()
#         break
        
# cv2.destroyAllWindows()

# cv2.waitKey(1)

In [4]:
# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(button):
    cap = cv2.VideoCapture(1)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)

            
# Run
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

[ WARN:0@28.866] global /tmp/pip-wheel-8c7uejek/opencv-python_88dbbad412c5416b992ae69de26299d6/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video1): can't open camera by index


None

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2832/2413573139.py", line 22, in view
    _, frame = cv2.imencode('.jpeg', frame)
cv2.error: OpenCV(4.6.0) /tmp/pip-wheel-8c7uejek/opencv-python_88dbbad412c5416b992ae69de26299d6/opencv/modules/imgcodecs/src/loadsave.cpp:976: error: (-215:Assertion failed) !image.empty() in function 'imencode'




In [7]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # frame = cv2.resize(frame, None, fx =0.5, fy=0.5, interpolation = cv2.INTER_AREA)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()